In [1]:
import pandas as pd
import sys

sys.path.append('./../../')
from src.data.conn_oracle import connect_oracle_trusted, connect_oracle_refined


In [ ]:
conn

In [2]:
# conn_trusted = connect_oracle_trusted()
conn_trusted = connect_oracle_refined()

User: TRIMBOX_REFINED
Host: ORACLE10G
Port: 1521
Service: adami.adami.corporativo


In [3]:
tables_disponiveis = pd.read_sql('SELECT table_name FROM user_tables ORDER BY table_name',con  =conn_trusted)
display(tables_disponiveis)

/tmp/ipykernel_1867965/1826810732.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_disponiveis = pd.read_sql('SELECT table_name FROM user_tables ORDER BY table_name',con  =conn_trusted)


,TABLE_NAME
0,CLIENTES
1,DAG_HEARTBEAT
2,FACAS
3,ITENS
4,MAQUINA
5,PARADAS
6,PEDIDOS
7,TAREFCON
8,TESTE


In [ ]:
df_final = pd.read_sql("""
                       SELECT 
    -- Identificadores principais
    t.ITEM,
    t.PEDIDO,
    t.MAQUINA,
    t.OP,
    t.OPONDULADA,
    
    -- Temporalidade
    t.INICIO,
    t.FIM,
    t.DURACAO,
    t.TURMA,
    t.DIADATURMA,
    
    -- Métricas de produção
    t.QUANTIDADEPROGRAMADA,
    t.QUANTIDADEPRODUZIDA,
    t.CHAPASALIMENTADAS,
    t.QUANTIDADEAJUSTE,
    t.GRAMATURA as GRAMATURA_TAREFCON,
    
    -- Classificação do evento
    t.FLAGPARADA,
    t.CODIGOPARADAOUCONV,
    t.DESCORIGEMREGISTRO,
    
    -- Controles operacionais
    t.REPROGRAMACAO,
    t.ARRANJO,
    t.USUARIO,
    t.FACA,
    t.SKIPFEED,
    
    -- Métricas calculadas (KPIs)
    CASE 
        WHEN t.QUANTIDADEPROGRAMADA > 0 THEN 
            ROUND((t.QUANTIDADEPRODUZIDA / t.QUANTIDADEPROGRAMADA) * 100, 2) 
        ELSE NULL 
    END as EFICIENCIA_PRODUCAO,
    
    CASE 
        WHEN t.DURACAO > 0 THEN 
            ROUND(t.QUANTIDADEPRODUZIDA / t.DURACAO, 4)
        ELSE NULL
    END as PRODUTIVIDADE_PECAS_MIN,  -- ADICIONADO: métrica principal
    
    CASE 
        WHEN t.FLAGPARADA = 1 AND par.USADACONVERSAO = 'S' THEN 'CONVERSAO' 
        WHEN t.FLAGPARADA = 1 AND (par.USADACONVERSAO = 'N' OR par.USADACONVERSAO IS NULL) THEN 'PARADA'
        ELSE 'PRODUCAO' 
    END as TIPO_REGISTRO,
    
    CASE
        WHEN t.FLAGPARADA = 0 THEN 'PRODUCAO NORMAL'
        WHEN t.FLAGPARADA = 1 AND par.USADACONVERSAO = 'S' THEN 'SETUP/CONVERSAO'
        WHEN t.FLAGPARADA = 1 THEN par.DESCRICAO
        ELSE 'OUTRO'
    END as CLASSIFICACAO_PARADA,
    
    -- Da tabela PEDIDOS
    p.IDCLIENTE,
    p.STATUSPEDIDO,
    p.DESCRSTATUSPEDIDO,
    p.EXIGELAUDO as EXIGELAUDO_PEDIDO,
    p.IDTIPOFT2 as IDTIPOFT2_PEDIDO,
    p.GRAMATURA as GRAMATURA_PEDIDO,
    p.LARGURAINTERNA,
    p.COMPRIMENTOINTERNO,
    p.ALTURAINTERNA,
    p.LARGPECA,
    p.COMPPECA,
    p.DESCRTIPODOPEDIDO,
    p.ARRANJO as ARRANJO_PEDIDO,
    p.CHAPA,
    p.COMPOSICAO,
    
    -- Da tabela ITENS
    i.ESTADOFT_DETEC,
    i.TEXTOESTADOFT_DETEC,
    i.STATUSFT,
    i.TEXTOSTATUSFT,
    i.IDFAMILIA,
    i.IDTIPOFT2 as IDTIPOFT2_ITEM,
    i.REFILADO,
    i.RESINAINTERNA,
    i.AMARRADO,
    i.PALETIZADO,
    i.PECASPORPACOTE,
    i.PECASPORPALETE,
    i.PACOTESPORPALETE,
    i.TIPOABNT,
    
    -- Da tabela MAQUINA
    m.TIPO as TIPO_MAQUINA,
    m.NRDECORES,
    m.IDGRUPOMAQUINA,
    
    -- Da tabela FACAS
    f.STATUS as STATUS_FACA,
    f.COMPLAMINA as COMPRIMENTO_LAMINA,
    
    -- Da tabela PARADAS
    par.DESCRICAO as DESCRICAO_PARADA,
    par.USADACONVERSAO as PARADA_CONVERSAO,
    par.PARADA as ID_PARADA,
    
    -- Da tabela CLIENTES
    c.CLIENTE,
    c.CODCLIENTE,
    
    -- Campos temporais calculados
    TO_CHAR(t.INICIO, 'YYYY-MM-DD') as DATA_INICIO,
    TO_CHAR(t.INICIO, 'HH24:MI:SS') as HORA_INICIO,
    TO_CHAR(t.FIM, 'YYYY-MM-DD') as DATA_FIM,
    TO_CHAR(t.FIM, 'HH24:MI:SS') as HORA_FIM,
    TO_NUMBER(TO_CHAR(t.INICIO, 'HH24')) as HORA_DO_DIA,
    TO_CHAR(t.INICIO, 'D') as DIA_DA_SEMANA,  -- 1=Dom, 7=Sab
    TO_CHAR(t.INICIO, 'FMDay', 'NLS_DATE_LANGUAGE=PORTUGUESE') as DIA_SEMANA_TEXTO,
    TO_NUMBER(TO_CHAR(t.INICIO, 'MM')) as MES,
    TO_NUMBER(TO_CHAR(t.INICIO, 'YYYY')) as ANO,
    TO_NUMB"""  """ER(TO_CHAR(t.INICIO, 'Q')) as TRIMESTRE

FROM TAREFCON t

-- Joins dimensionais
LEFT JOIN PEDIDOS p 
    ON t.PEDIDO = p.PEDIDO 
    AND t.ITEM = p.ITEM  -- CRÍTICO: chave composta

LEFT JOIN ITENS i 
    ON t.ITEM = i.ITEM

LEFT JOIN MAQUINA m 
    ON t.MAQUINA = m.MAQUINA

LEFT JOIN FACAS f 
    ON t.FACA = f.CODFACA

LEFT JOIN PARADAS par 
    ON t.CODIGOPARADAOUCONV = par.PARADA 

LEFT JOIN CLIENTES c 
    ON p.IDCLIENTE = c.IDCLIENTE  -- CORRIGIDO: apenas p.IDCLIENTE existe

-- Filtros de qualidade
WHERE 
    t.INICIO IS NOT NULL
    AND t.DURACAO IS NOT NULL
    AND t.DURACAO > 0  -- Evitar divisão por zero em cálculos de produtividade
    AND (p.STATUSPEDIDO IS NULL OR p.STATUSPEDIDO <> 4)  -- Excluir cancelados
""", con=conn_trusted)

/tmp/ipykernel_328723/3269174702.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_final = pd.read_sql("""


In [4]:
df_final.shape

(1604174, 76)

In [5]:
df_final.to_parquet(
    "../../data/02 - trusted/tb_query.parquet"
)


In [8]:
df = df_final.copy()

In [9]:
import pandas as pd
import numpy as np

# Assumindo que você já tem:
# df = pd.read_sql(query, connection)
# df_tarefcon = pd.read_sql("SELECT * FROM TAREFCON", connection)

# ============================================================
# 1. VERIFICAR SE HÁ DUPLICAÇÃO
# ============================================================

print("=" * 60)
print("VERIFICAÇÃO DE DUPLICAÇÃO")
print("=" * 60)

# Contar registros no df final
total_df_final = len(df_final)
print(f"Total de registros no DF final: {total_df_final:,}")

# Contar registros em TAREFCON com os mesmos filtros
df_tarefcon_filtrado = df_tarefcon[
    (df_tarefcon['INICIO'].notna()) & 
    (df_tarefcon['DURACAO'].notna()) &
    (df_tarefcon['DURACAO'] > 0)
]
total_tarefcon_filtrado = len(df_tarefcon_filtrado)
print(f"Total de registros em TAREFCON (filtrado): {total_tarefcon_filtrado:,}")

# Verificar diferença
diferenca = total_df_final - total_tarefcon_filtrado
print(f"\nDiferença: {diferenca:,}")

if diferenca > 0:
    print(f"⚠️ ATENÇÃO: Há {diferenca:,} registros A MAIS no DF final!")
    print("   Possível causa: duplicação nos JOINs (problema de cardinalidade)")
elif diferenca < 0:
    print(f"⚠️ ATENÇÃO: Há {abs(diferenca):,} registros A MENOS no DF final!")
    print("   Possível causa: filtros adicionais ou registros excluídos")
else:
    print("✓ OK: Mesma quantidade de registros (sem duplicação)")

# Verificar duplicatas por chave primária esperada
chave_primaria = ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA']
duplicatas = df[df.duplicated(subset=chave_primaria, keep=False)]

if len(duplicatas) > 0:
    print(f"\n⚠️ ATENÇÃO: {len(duplicatas):,} registros duplicados detectados!")
    print("\nAmostra de duplicatas:")
    print(duplicatas[chave_primaria].head(10))
else:
    print("\n✓ OK: Nenhuma duplicata encontrada na chave primária")

# ============================================================
# 2. VERIFICAR MATCH RATE DOS JOINS
# ============================================================

print("\n" + "=" * 60)
print("MATCH RATE DOS JOINS")
print("=" * 60)

total = len(df)

# Match com PEDIDOS
com_pedido = df['PEDIDO'].notna().sum()
pct_pedido = (com_pedido / total) * 100
print(f"\nPEDIDOS:")
print(f"  - Com match: {com_pedido:,} ({pct_pedido:.2f}%)")
print(f"  - Sem match: {total - com_pedido:,} ({100 - pct_pedido:.2f}%)")

# Match com ITENS
com_item = df['ESTADOFT_DETEC'].notna().sum()  # Usar campo exclusivo de ITENS
pct_item = (com_item / total) * 100
print(f"\nITENS:")
print(f"  - Com match: {com_item:,} ({pct_item:.2f}%)")
print(f"  - Sem match: {total - com_item:,} ({100 - pct_item:.2f}%)")

# Match com MAQUINA
com_maquina = df['TIPO_MAQUINA'].notna().sum()  # Usar campo exclusivo de MAQUINA
pct_maquina = (com_maquina / total) * 100
print(f"\nMAQUINA:")
print(f"  - Com match: {com_maquina:,} ({pct_maquina:.2f}%)")
print(f"  - Sem match: {total - com_maquina:,} ({100 - pct_maquina:.2f}%)")

# Match com FACAS (nem sempre obrigatório)
com_faca = df['STATUS_FACA'].notna().sum()
pct_faca = (com_faca / total) * 100
print(f"\nFACAS:")
print(f"  - Com match: {com_faca:,} ({pct_faca:.2f}%)")
print(f"  - Sem match: {total - com_faca:,} ({100 - pct_faca:.2f}%)")
print(f"  - Obs: Facas não são obrigatórias (tipo Flexo não usa)")

# Match com PARADAS (apenas quando FLAGPARADA = 1)
registros_parada = df[df['FLAGPARADA'] == 1]
com_parada = registros_parada['DESCRICAO_PARADA'].notna().sum()
total_paradas = len(registros_parada)
if total_paradas > 0:
    pct_parada = (com_parada / total_paradas) * 100
    print(f"\nPARADAS (apenas registros com FLAGPARADA=1):")
    print(f"  - Total de paradas: {total_paradas:,}")
    print(f"  - Com descrição: {com_parada:,} ({pct_parada:.2f}%)")
    print(f"  - Sem descrição: {total_paradas - com_parada:,} ({100 - pct_parada:.2f}%)")

# Match com CLIENTES (via PEDIDOS)
com_cliente = df['CLIENTE'].notna().sum()
pct_cliente = (com_cliente / total) * 100
print(f"\nCLIENTES:")
print(f"  - Com match: {com_cliente:,} ({pct_cliente:.2f}%)")
print(f"  - Sem match: {total - com_cliente:,} ({100 - pct_cliente:.2f}%)")

# ============================================================
# 3. RESUMO CONSOLIDADO
# ============================================================

print("\n" + "=" * 60)
print("RESUMO CONSOLIDADO")
print("=" * 60)

resumo = pd.DataFrame({
    'Tabela': ['PEDIDOS', 'ITENS', 'MAQUINA', 'FACAS', 'CLIENTES'],
    'Match': [com_pedido, com_item, com_maquina, com_faca, com_cliente],
    'Match (%)': [pct_pedido, pct_item, pct_maquina, pct_faca, pct_cliente],
    'Sem Match': [total - com_pedido, total - com_item, total - com_maquina, 
                  total - com_faca, total - com_cliente],
    'Sem Match (%)': [100 - pct_pedido, 100 - pct_item, 100 - pct_maquina,
                      100 - pct_faca, 100 - pct_cliente]
})

print(resumo.to_string(index=False))

# ============================================================
# 4. ANÁLISE DE QUALIDADE ADICIONAL
# ============================================================

print("\n" + "=" * 60)
print("ANÁLISE DE QUALIDADE ADICIONAL")
print("=" * 60)

# Verificar consistência de gramatura
if 'GRAMATURA_TAREFCON' in df.columns and 'GRAMATURA_PEDIDO' in df.columns:
    df_com_ambas = df[(df['GRAMATURA_TAREFCON'].notna()) & (df['GRAMATURA_PEDIDO'].notna())]
    divergencias = df_com_ambas[
        abs(df_com_ambas['GRAMATURA_TAREFCON'] - df_com_ambas['GRAMATURA_PEDIDO']) > 10
    ]
    print(f"\nDivergências de GRAMATURA (diferença > 10):")
    print(f"  - Total com ambas gramaturas: {len(df_com_ambas):,}")
    print(f"  - Com divergência: {len(divergencias):,} ({(len(divergencias)/len(df_com_ambas)*100):.2f}%)")

# Verificar valores nulos em campos críticos
print(f"\nValores nulos em campos críticos:")
campos_criticos = ['ITEM', 'MAQUINA', 'INICIO', 'FIM', 'DURACAO', 'QUANTIDADEPRODUZIDA']
for campo in campos_criticos:
    if campo in df.columns:
        nulos = df[campo].isna().sum()
        pct = (nulos / total) * 100
        print(f"  - {campo}: {nulos:,} ({pct:.2f}%)")

# Verificar distribuição por tipo de registro
print(f"\nDistribuição por TIPO_REGISTRO:")
if 'TIPO_REGISTRO' in df.columns:
    dist = df['TIPO_REGISTRO'].value_counts()
    for tipo, count in dist.items():
        pct = (count / total) * 100
        print(f"  - {tipo}: {count:,} ({pct:.2f}%)")

print("\n" + "=" * 60)

VERIFICAÇÃO DE DUPLICAÇÃO
Total de registros no DF final: 1,604,174
Total de registros em TAREFCON (filtrado): 1,604,242

Diferença: -68
⚠️ ATENÇÃO: Há 68 registros A MENOS no DF final!
   Possível causa: filtros adicionais ou registros excluídos

⚠️ ATENÇÃO: 1,288,367 registros duplicados detectados!

Amostra de duplicatas:
    ITEM PEDIDO     INICIO        FIM TURMA MAQUINA
7   None   None 2022-06-27 2022-06-27     C     ACB
8   None   None 2022-06-27 2022-06-27     C     ACB
9   None   None 2022-06-27 2022-06-27     C     ACB
11  None   None 2022-06-28 2022-06-28     C     ACB
14  None   None 2022-06-28 2022-06-28     C     ACB
16  None   None 2022-07-04 2022-07-04     C     ACB
17  None   None 2022-07-04 2022-07-04     C     ACB
18  None   None 2022-07-04 2022-07-04     C     ACB
24  None   None 2022-07-07 2022-07-07     C     ACB
26  None   None 2022-07-07 2022-07-07     C     ACB

MATCH RATE DOS JOINS

PEDIDOS:
  - Com match: 319,284 (19.90%)
  - Sem match: 1,284,890 (80.10%)

IT

In [11]:
import pandas as pd

print("=" * 60)
print("INVESTIGAÇÃO DA DUPLICAÇÃO")
print("=" * 60)

# 1. Verificar se INICIO e FIM têm componente de tempo
print("\n1. Verificar timestamps em INICIO e FIM:")
print("Amostra de INICIO:")
print(df['INICIO'].head(20))
print("\nTipo de dados:")
print(df['INICIO'].dtype)

# Verificar se há informação de hora
df['tem_hora'] = df['INICIO'].astype(str).str.contains(r'\d{2}:\d{2}:\d{2}')
print(f"\nRegistros com hora no INICIO: {df['tem_hora'].sum():,} ({df['tem_hora'].sum()/len(df)*100:.1f}%)")

# 2. Verificar padrão de duplicação
print("\n" + "=" * 60)
print("2. Análise de registros duplicados:")

# Identificar chave real tentando adicionar mais campos
chaves_testar = [
    ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA'],
    ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 'CODIGOPARADAOUCONV'],
    ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 'FLAGPARADA', 'CODIGOPARADAOUCONV'],
    ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 'OP'],
]

for i, chave in enumerate(chaves_testar):
    # Filtrar apenas colunas que existem
    chave_existente = [c for c in chave if c in df.columns]
    n_dup = df.duplicated(subset=chave_existente, keep=False).sum()
    print(f"Chave {i+1} {chave_existente}: {n_dup:,} duplicatas")

# 3. Analisar especificamente registros com ITEM/PEDIDO nulos
print("\n" + "=" * 60)
print("3. Análise de registros SEM ITEM/PEDIDO (paradas):")

df_sem_item = df[df['ITEM'].isna()]
print(f"Total de registros sem ITEM: {len(df_sem_item):,}")
print(f"Percentual do dataset: {len(df_sem_item)/len(df)*100:.2f}%")

# Verificar se esses são paradas
if 'FLAGPARADA' in df.columns:
    paradas = df_sem_item['FLAGPARADA'].value_counts()
    print(f"\nDistribuição de FLAGPARADA nestes registros:")
    print(paradas)

# Verificar quantos são únicos considerando CODIGOPARADAOUCONV
chave_parada = ['INICIO', 'FIM', 'TURMA', 'MAQUINA', 'CODIGOPARADAOUCONV']
chave_parada_existente = [c for c in chave_parada if c in df.columns]
n_unicos_parada = df_sem_item.drop_duplicates(subset=chave_parada_existente).shape[0]
print(f"\nRegistros únicos usando chave de parada {chave_parada_existente}: {n_unicos_parada:,}")

# 4. Analisar um dia específico com duplicatas
print("\n" + "=" * 60)
print("4. Exemplo detalhado de um dia com duplicatas:")

# Pegar um dia com muitas duplicatas
dia_exemplo = df_sem_item.groupby('INICIO').size().sort_values(ascending=False).index[0]
exemplo = df_sem_item[df_sem_item['INICIO'] == dia_exemplo].head(20)

print(f"\nDia: {dia_exemplo}")
print(f"Total de registros neste dia: {len(df_sem_item[df_sem_item['INICIO'] == dia_exemplo]):,}")
print("\nAmostra detalhada:")
cols_mostrar = ['INICIO', 'FIM', 'TURMA', 'MAQUINA', 'FLAGPARADA', 
                'CODIGOPARADAOUCONV', 'DESCRICAO_PARADA', 'DURACAO']
cols_existentes = [c for c in cols_mostrar if c in exemplo.columns]
print(exemplo[cols_existentes].to_string())

# 5. Verificar se há algum ID único oculto
print("\n" + "=" * 60)
print("5. Procurar possível chave única:")

# Testar combinações mais complexas
if 'OP' in df.columns and 'CODIGOPARADAOUCONV' in df.columns:
    chave_completa = ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 
                      'OP', 'FLAGPARADA', 'CODIGOPARADAOUCONV', 'ARRANJO']
    chave_completa_existente = [c for c in chave_completa if c in df.columns]
    n_dup_completa = df.duplicated(subset=chave_completa_existente, keep=False).sum()
    print(f"Duplicatas com chave expandida {chave_completa_existente}: {n_dup_completa:,}")


INVESTIGAÇÃO DA DUPLICAÇÃO

1. Verificar timestamps em INICIO e FIM:
Amostra de INICIO:
0    2022-06-20
1    2022-06-21
2    2022-06-22
3    2022-06-22
4    2022-06-22
5    2022-06-27
6    2022-06-27
7    2022-06-27
8    2022-06-27
9    2022-06-27
10   2022-06-28
11   2022-06-28
12   2022-06-28
13   2022-06-28
14   2022-06-28
15   2022-07-04
16   2022-07-04
17   2022-07-04
18   2022-07-04
19   2022-07-06
Name: INICIO, dtype: datetime64[ns]

Tipo de dados:
datetime64[ns]

Registros com hora no INICIO: 0 (0.0%)

2. Análise de registros duplicados:
Chave 1 ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA']: 1,288,367 duplicatas
Chave 2 ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 'CODIGOPARADAOUCONV']: 1,143,943 duplicatas
Chave 3 ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 'FLAGPARADA', 'CODIGOPARADAOUCONV']: 1,143,943 duplicatas
Chave 4 ['ITEM', 'PEDIDO', 'INICIO', 'FIM', 'TURMA', 'MAQUINA', 'OP']: 1,229,387 duplicatas

3. Análise de registros SEM ITEM/PEDIDO

### Facas

In [4]:
df_facas = pd.read_sql('SELECT CODFACA,STATUS,COMPLAMINA,DESATIVADOSN FROM FACAS', con = conn_trusted)

/tmp/ipykernel_380413/409803162.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_facas = pd.read_sql('SELECT CODFACA,STATUS,COMPLAMINA,DESATIVADOSN FROM FACAS', con = conn_trusted)


In [18]:
df_facas.to_parquet(
    "../../data/02 - trusted/tb_facas.parquet"
)


### CLIENTES

In [5]:
df_clientes = pd.read_sql('SELECT  IDCLIENTE, CLIENTE, CODCLIENTE,CODREPRESENTANTE FROM CLIENTES', con = conn_trusted)

/tmp/ipykernel_380413/14337810.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_clientes = pd.read_sql('SELECT  IDCLIENTE, CLIENTE, CODCLIENTE,CODREPRESENTANTE FROM CLIENTES', con = conn_trusted)


In [17]:
df_clientes.to_parquet(
    "../../data/02 - trusted/tb_clientes.parquet"
)


### ITENS

In [16]:
query_itens = """
SELECT
    ITEM,
    IDTIPOFT2,
    IDFAMILIA,
    ESTADOFT_DETEC,
    TEXTOESTADOFT_DETEC,
    STATUSFT,
    TEXTOSTATUSFT,
    IDCLIENTE,
    REFERENCIA,
    CODIGOREFERENCIA,
    TIPOABNT,
    EXIGELAUDO,
    GRAMATURA,
    COLUNAMINIMO,
    COBBINTMAXIMO,
    COMPRESSAO,
    COMPOSICAO,
    LARGURA,
    REFILOLARGURA,
    COMPRIMENTO,
    REFILOCOMPRIMENTO,
    MULTLARG,
    MULTCOMP,
    ARRANJO,
    REFUGOCLIENTE,
    VINCOLARG1,
    VINCOLARG2,
    VINCOLARG3,
    VINCOCOMP1,
    VINCOCOMP2,
    VINCOCOMP3,
    VINCOCOMP4,
    VINCOCOMP5,
    LAP,
    PROLONGLAP,
    LAPNOCOMP,
    LAPINTERNO,
    REFILADO,
    RESINAINTERNA,
    AMARRADO,
    PALETIZADO,
    PACOTESLARGURA,
    PACOTESCOMPRIMENTO,
    PACOTESALTURA,
    PECASPORPACOTE,
    PECASPORPALETE,
    PACOTESPORPALETE,
    UNIDADESPORPALETE,
    ESPELHO,
    FILME,
    FACA,
    COR1,
    CONSUMOCOR1,
    COR2,
    CONSUMOCOR2,
    COR3,
    CONSUMOCOR3,
    COR4,
    CONSUMOCOR4,
    NRCORES,
    LARGURAINTERNA,
    COMPRIMENTOINTERNO,
    ALTURAINTERNA,
    LARGPECA,
    COMPPECA,
    COMPPACOTE,
    LARGPACOTE,
    ALTURAPACOTE,
    COMMPALETEFECHADO,
    LARGPALETEFECHADO,
    ALTURAPALETEFECHADO,
    PESOCAIXA,
    PATHFIGURADOLASTRO,
    IDPALETE,
    AREABRUTAPECACOMREFILOS,
    AREABRUTAPECA,
    AREALIQUIDAPECA,
    AREABRUTACHAPA,
    AREALIQUIDACHAPA,
    VOLUMEPALETEFECHADOM3,
    VOLUMEPACOTEFECHADOM3
FROM ITENS

"""
df_itens = pd.read_sql(query_itens, con = conn_trusted)


/tmp/ipykernel_380413/2398359529.py:87: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_itens = pd.read_sql(query_itens, con = conn_trusted)


In [15]:
df_itens.to_parquet(
    "../../data/02 - trusted/tb_itens.parquet"
)


### MAQUINA

In [20]:
df_maquina = pd.read_sql('SELECT MAQUINA, TIPO, NRDECORES,IDGRUPOMAQUINA FROM MAQUINA', con = conn_trusted)
df_maquina

/tmp/ipykernel_380413/2676442152.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_maquina = pd.read_sql('SELECT MAQUINA, TIPO, NRDECORES,IDGRUPOMAQUINA FROM MAQUINA', con = conn_trusted)


,MAQUINA,TIPO,NRDECORES,IDGRUPOMAQUINA
0,ACB,7,NaN,NaN
1,BALPIC,8,NaN,NaN
2,BAY1,2,0.0,5.0
3,CAUT,5,NaN,NaN
4,CAUT2,5,NaN,17.0
5,CAUT3,5,0.0,14.0
6,CHAPA,6,NaN,20.0
7,CHP,6,NaN,20.0
8,CMAN,5,NaN,15.0
9,CUR1,3,2.0,NaN


In [21]:
df_maquina.to_parquet(
    "../../data/02 - trusted/tb_maquina.parquet"
)


### PARADAS

In [24]:
df_paradas = pd.read_sql('SELECT PARADA,DESCRICAO,USADACONVERSAO,DESATIVADA,FLAGEXTERNA FROM PARADAS', con=conn_trusted)
df_paradas


/tmp/ipykernel_380413/1217593623.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_paradas = pd.read_sql('SELECT PARADA,DESCRICAO,USADACONVERSAO,DESATIVADA,FLAGEXTERNA FROM PARADAS', con=conn_trusted)


,PARADA,DESCRICAO,USADACONVERSAO,DESATIVADA,FLAGEXTERNA
0,245,ENROLOU PAPEL NA PRENSA,N,N,N
1,246,ENROLOU PAPEL NO ROLO CORRUGAD,N,N,N
2,247,EXCESSO DE AGUA NA COLA,N,N,N
3,248,EXCESSO DE FREIO,N,N,N
4,249,FALHA OPERACIONAL,N,N,N
...,...,...,...,...,...
694,11101,AVARIA MECÂNICA,N,N,N
695,11102,BLOQUEIO MECÂNICO,N,N,N
696,11103,MANCAL AVARIADO,N,N,N
697,11104,CORRENTE AVARIADA,N,N,N


In [25]:
df_paradas.to_parquet(
    "../../data/02 - trusted/tb_paradas.parquet"
)


### PEDIDOS

In [27]:
query_pedidos = """
SELECT
    Pedido,
    Item,
    StatusPedido,
    DescrStatusPedido,
    Suspenso,
    SuspOuCancel,
    IDCliente,
    Referencia,
    CodigoReferencia,
    TipoABNT,
    idTipoFT2,
    ExigeLaudo,
    Gramatura,
    ColunaMinimo,
    CobbIntMaximo,
    Compressao,
    Chapa,
    Composicao,
    Largura,
    RefiloLargura,
    Comprimento,
    RefiloComprimento,
    MultLarg,
    MultComp,
    Arranjo,
    RefugoCliente,
    PesoPeca,
    VincoLarg1,
    VincoLarg2,
    VincoLarg3,
    VincoComp1,
    VincoComp2,
    VincoComp3,
    VincoComp4,
    VincoComp5,
    Lap,
    ProlongLap,
    LapNoComp,
    LapInterno,
    Refilado,
    ResinaInterna,
    Amarrado,
    Paletizado,
    PacotesLargura,
    PACOTOSCOMPRIMENTO,
    PacotesAltura,
    PecasPorPacote,
    PecasPorPalete,
    UnidadesPorPalete,
    PacotesPorPalete,
    Espelho,
    Filme,
    Faca,
    Cor1,
    ConsumoCor1,
    Cor2,
    ConsumoCor2,
    Cor3,
    ConsumoCor3,
    Cor4,
    ConsumoCor4,
    NrCores,
    DataEntrega2,
    QuantidadePedida,
    QuantidadePedidaMin,
    QuantidadePedidaMax,
    TipoEntrega,
    DescTipoEntrega,
    LarguraInterna,
    ComprimentoInterno,
    AlturaInterna,
    LargPeca,
    CompPeca,
    DataEntregaOriginal,
    DescrTipoDoPedido,
    CompPacote,
    LargPacote,
    AlturaPacote,
    CommPaleteFechado,
    LargPaleteFechado,
    AlturaPaleteFechado,
    PesoCaixa,
    PathFiguraDoLastro,
    IDPalete,
    AreaBrutaPecaComRefilos,
    AreaBrutaPeca,
    AreaLiquidaPeca,
    AreaBrutaChapa,
    AreaLiquidaChapa,
    VolumePaleteFechadoM3,
    VolumePacoteFechadoM3,
    VolumeFechadoPedido
FROM Pedidos

"""
df_pedidos = pd.read_sql(query_pedidos, con = conn_trusted)
df_pedidos

/tmp/ipykernel_380413/720560523.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pedidos = pd.read_sql(query_pedidos, con = conn_trusted)


,PEDIDO,ITEM,STATUSPEDIDO,DESCRSTATUSPEDIDO,SUSPENSO,SUSPOUCANCEL,IDCLIENTE,REFERENCIA,CODIGOREFERENCIA,TIPOABNT,...,PATHFIGURADOLASTRO,IDPALETE,AREABRUTAPECACOMREFILOS,AREABRUTAPECA,AREALIQUIDAPECA,AREABRUTACHAPA,AREALIQUIDACHAPA,VOLUMEPALETEFECHADOM3,VOLUMEPACOTEFECHADOM3,VOLUMEFECHADOPEDIDO
0,584184-8,658931,3,3-Expedido,0,Não,1840,433154-CPO-0300-FDO-MA-V70 (AVES),433154,19,...,12.bmp,9.0,0.483702,0.469845,0.462327,1.934810,1.879380,1.267200,0.068128,15.206400
1,584184-9,658931,3,3-Expedido,0,Não,1840,433154-CPO-0300-FDO-MA-V70 (AVES),433154,19,...,12.bmp,9.0,0.483702,0.469845,0.462327,1.934810,1.879380,1.267200,0.068128,15.206400
2,582746-13,635790,3,3-Expedido,0,Não,11833,220688-V05,220688,27,...,12.bmp,50.0,0.419936,0.410366,0.397234,1.679743,1.641464,2.043720,0.061555,32.699520
3,582746-14,635790,3,3-Expedido,0,Não,11833,220688-V05,220688,27,...,12.bmp,50.0,0.424018,0.410366,0.397234,1.696071,1.641464,2.043720,0.061555,32.699520
4,582746-19,635790,3,3-Expedido,0,Não,11833,220688-V05,220688,27,...,12.bmp,50.0,0.424018,0.410366,0.397234,1.696071,1.641464,2.043720,0.061555,32.699520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384528,PRD092391,670682,1,1-Ativo,0,Não,11572,013406 - CALÇO TABULEIRO VENT. COLUNA 50,013406,29,...,22.BMP,24.0,0.240100,0.240100,0.235298,0.960400,0.960400,1.548000,0.034094,24.768000
384529,PRD092392,889540,1,1-Ativo,0,Não,1824,4002734-CPO-0201-SA-B-02-V-70 (Industrializados),4002734,CNAR,...,32.bmp,16.0,0.386325,0.386325,0.377469,0.386325,0.386325,2.237625,0.028114,26.243611
384530,PRD092470,868580,1,1-Ativo,0,Não,9642,72784-V00/GP-Rv.4/TER,72784,CN,...,12.bmp,15.0,0.878880,0.878880,0.861708,0.878880,0.878880,1.695760,0.106941,67.830400
384531,PRD092471,870290,1,1-Ativo,0,Não,1895,72784-V00/GP-Rv.4,72784,CN,...,1+2-DOURADOS.bmp,42.0,0.878880,0.878880,0.861708,0.878880,0.878880,2.719776,0.106941,174.065664


In [28]:
df_pedidos.to_parquet(
    "../../data/02 - trusted/tb_pedidos.parquet"
)


### TAREFCON

In [6]:
query_tarefcon = """ 
SELECT
    Maquina,
    FlagParada,
    CodigoParadaOuConv,
    Turma,
    OP,
    Pedido,
    Item,
    Reprogramacao,
    Arranjo,
    Gramatura,
    QuantidadeProgramada,
    ChapasAlimentadas,
    QuantidadeProduzida,
    QuantidadeAjuste,
    DuracaoPrevista,
    Inicio,
    Fim,
    DiaDaTurma,
    IDCliente,
    Usuario,
    OrigemRegistro,
    DescOrigemRegistro,
    SkipFeed,
    OPOndulada,
    Duracao,
    Faca
FROM TarefCon
"""


In [7]:

df_tarefcon = pd.read_sql(query_tarefcon, con = conn_trusted)
df_tarefcon

/tmp/ipykernel_328723/1265925900.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tarefcon = pd.read_sql(query_tarefcon, con = conn_trusted)


,MAQUINA,FLAGPARADA,CODIGOPARADAOUCONV,TURMA,OP,PEDIDO,ITEM,REPROGRAMACAO,ARRANJO,GRAMATURA,...,FIM,DIADATURMA,IDCLIENTE,USUARIO,ORIGEMREGISTRO,DESCORIGEMREGISTRO,SKIPFEED,OPONDULADA,DURACAO,FACA
0,ACB,0,NaN,C,598850-2/659652,598850-2,659652,1.0,1.0,364.0,...,2022-06-21,2022-06-20,1428.0,191.0,0.0,Manual padrão,0,None,180,6111-2
1,ACB,0,NaN,C,598850-2/659652,598850-2,659652,1.0,1.0,364.0,...,2022-06-21,2022-06-21,1428.0,191.0,0.0,Manual padrão,0,None,60,6111-2
2,ACB,0,NaN,B,598850-2/659652,598850-2,659652,1.0,1.0,364.0,...,2022-06-22,2022-06-22,1428.0,630.0,0.0,Manual padrão,0,None,80,6111-2
3,ACB,0,NaN,B,598771-1/398692,598771-1,398692,1.0,1.0,475.0,...,2022-06-22,2022-06-22,9758.0,191.0,0.0,Manual padrão,0,None,251,4756
4,ACB,1,138.0,B,None,None,None,NaN,NaN,NaN,...,2022-06-22,2022-06-22,NaN,191.0,0.0,Manual padrão,0,None,149,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604557,WAR3,0,NaN,C,717003-6/350001,717003-6,350001,1.0,4.0,416.0,...,2025-08-05,2025-08-04,1774.0,445.0,1.0,Online CLP,0,PRD099638/Várias FTs,60,6844-1
1604558,WAR3,1,130.0,C,710976-28/798711,None,None,NaN,NaN,NaN,...,2025-08-05,2025-08-04,NaN,445.0,1.0,Online CLP,0,None,2,None
1604559,WAR3,1,140.0,C,710976-28/798711,None,None,NaN,NaN,NaN,...,2025-08-05,2025-08-04,NaN,445.0,1.0,Online CLP,0,None,1,None
1604560,WAR3,1,140.0,C,710976-28/798711,None,None,NaN,NaN,NaN,...,2025-08-05,2025-08-04,NaN,445.0,1.0,Online CLP,0,None,2,None


In [32]:
df_tarefcon.to_parquet(
    "../../data/02 - trusted/tb_tarefcon.parquet"
)
